## Lab 8
## Fundamental Actor-Critic Algorithms

#### Professor - Jeremy Curuksu, PhD (jeremy.cur@nyu.edu)

#### Section Leader - Anudeep Tubati (at5373@nyu.edu)
<hr>
<br>

In today's lab, we will learn to implement two essential algorithms from the Actor-Critic family: Advantage Actor Critic (https://openai.com/research/openai-baselines-acktr-a2c), and Deep Deterministic Policy Gradients (https://www.deepmind.com/publications/deterministic-policy-gradient-algorithms).

<br>

A2C helps in reducing the variance of the updates stemming from the empirical cumulative reward as seen in REINFORCE. We will see how it does so. Whereas, DDPG focuses on a different kind of problems, those with continuous action spaces. Can you think of a way to solve continuous action spaces using DQNs? 

### Part I
### Advantage Actor Critic

Recollect the policy gradient update from REINFORCE: $\sum_t \nabla G_t \log \pi(a_t | s_t)$

#### Using Q(s, a) and V(s)
Let's compute the advantage of an action defined by

<center>$A(s, a) = Q(s, a) - V(s)$</center>

Note: We may also subtract the average return of state s instead of using V(s). It is called using a baseline. However, it is easy to see that approximating V(s) non-linearly may be better than using a moving average.

#### Required Imports

In [25]:
# Advantage Actor-Critic Algorithm on Pong, Author: Anudeep Tubati, NYU
# Modified implementation of REINFORCE at https://karpathy.github.io/2016/05/31/rl/
import sys
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Bernoulli

import gymnasium as gym

from PIL import Image

from time import sleep

from pathlib import Path


# code for the only two actions in Pong
# 2 -> UP, 3 -> DOWN
ACTIONS = [2, 3]

#### Helper Functions

In [26]:
def prepro(img):
    """ prepro 210x160x3 uint8 frame into 6000 (75x80) 1D float vector """
    img = img[35:185]
    img = img[::2, ::2, 0]
    img[img == 144] = 0
    img[img == 109] = 0
    img[img != 0] = 1

    return img.astype(np.float32).ravel()

def discount_rewards(reward):
    # Compute the gamma-discounted rewards over an episode
    gamma = 0.99    # discount rate
    running_add = 0
    discounted_r = torch.zeros_like(reward)

    for i in reversed(range(0, len(reward))):
        if reward[i] != 0: # reset the sum, since this was a game boundary (pong specific!)
            running_add = 0
        running_add = running_add * gamma + reward[i]
        discounted_r[i] = running_add

    discounted_r -= torch.mean(discounted_r) # normalizing the result
    discounted_r /= torch.std(discounted_r) # divide by standard deviation
    return discounted_r


def log(filename, string):
    with open(filename, 'a+') as logger:
        logger.write(string)


# wrapper for the Gym environment that uses our helper functions
class Pong(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env

        self.ROWS = 75
        self.COLS = 80

        self.state_size = (self.ROWS * self.COLS, )

    def reset(self):
        self.prev_x = prepro(self.env.reset()[0])

        return np.zeros(self.state_size).flatten()
    
    def step(self, action):
        next_state, reward, done, _, info = self.env.step(action)

        next_state = prepro(next_state)
        actual_next_state = next_state - self.prev_x
        self.prev_x = next_state

        return actual_next_state.flatten(), reward, done, info

#### Agent and Networks

In [27]:
class Actor(nn.Module):
    def __init__(self, input_size, action_size):
        super(Actor, self).__init__()

        self.fc1 = nn.Linear(input_size, 200)
        self.output = nn.Linear(200, action_size)
    
    def forward(self, x):

        x = F.relu(self.fc1(x))
        action_prob = torch.sigmoid(self.output(x))

        return action_prob


class Critic(nn.Module):
    def __init__(self, input_size):
        super(Critic, self).__init__()

        self.fc1 = nn.Linear(input_size, 200)
        self.output = nn.Linear(200, 1)
    
    def forward(self, x):

        x = F.relu(self.fc1(x))
        value = self.output(x)

        return value


class A2CAgent(object):
    def __init__(self, input_size, log_filename):
        self.actor = Actor(input_size, 1)
        self.critic = Critic(input_size)
        self.optimizerA = optim.RMSprop(self.actor.parameters(), lr=0.001, weight_decay=0.99)
        self.optimizerC = optim.RMSprop(self.critic.parameters(), lr=0.001, weight_decay=0.99)

        self.memory = {
            'rewards': [],
            'log_probs': [],
            'states': []
        }
        self.epoch = 0

        self.log_filename = log_filename
    
    def select_action(self, state):

        # sample an action from stochastic policy
        action_prob = self.actor.forward(state)
        dist = Bernoulli(action_prob)

        sampled_val = dist.sample()
        action_idx = int(sampled_val.item())

        # compute log prob
        # print(sampled_val.item() == 1.0, sampled_val, action_idx)
        action_to_take = ACTIONS[action_idx]

        self.memory['log_probs'].append(dist.log_prob(sampled_val))

        return action_to_take

    def remember(self, state, reward):
        self.memory['states'].append(state)
        self.memory['rewards'].append(reward)
    
    def update_network(self):
        len_r = len(self.memory['rewards'])
        assert len_r == len(self.memory['log_probs'])

        # convert to tensors for ease of operation
        self.memory['rewards'] = torch.tensor(self.memory['rewards'], dtype=torch.float32)
        discounted_r = discount_rewards(self.memory['rewards']).unsqueeze(1)
        self.memory['log_probs'] = torch.stack(self.memory['log_probs'])
        states = torch.stack(self.memory['states'])

        # get V values of states from critic
        values = self.critic.forward(states)

        CHANGE_SCHED_EPOCH = 1000

        # train only critic first, then train policy as well
        if self.epoch <= CHANGE_SCHED_EPOCH:
            # calculate policy loss
            policy_losses = (-1 * self.memory['log_probs']) * discounted_r

            # calculate loss for critic
            value_loss = F.mse_loss(values, discounted_r)

        else:
            if self.epoch == (CHANGE_SCHED_EPOCH + 9):
                self.optimizerA.param_groups[0]['lr'] = 0.0005
                print("\nACTOR LR CHANGED TO 0.0005")

            # calculate advantages for A2C
            advantages = discounted_r - values.detach()

            # calculate policy loss
            policy_losses = (-1 * self.memory['log_probs']) * advantages

            # calculate targets for critic by adding discounted next_state
            # values (except for last state)
            targets = self.memory['rewards'].unsqueeze(1).clone()
            targets[:-1] += (0.99 * values.detach())[1:]

            # calculate value loss from targets
            value_loss = F.mse_loss(values, targets)

        # print(f"[{self.epoch}]", value_loss)

        # crux of training
        self.optimizerA.zero_grad()
        self.lossA = policy_losses.sum()
        self.lossA.backward()
        self.optimizerA.step()

        self.optimizerC.zero_grad()
        self.lossC = value_loss
        self.lossC.backward()
        self.optimizerC.step()

        # reset memory because this is on-policy
        for k in self.memory.keys():
            self.memory[k] = []
    
    def learn(self, env, num_epochs, roll_size, start=0):

        print(f"Resuming from {start + 1}, Writing to {self.log_filename}\n")
        # self.log_file.write(f"Resuming from {start + 1}\n\n")

        assert roll_size == 10

        avg = -float('inf')
        best_avg = -float('inf')
        max_score = -float('inf')
        all_scores = np.zeros((num_epochs, ), dtype=np.int32)

        for eps_idx in range(start + 1, num_epochs):
            self.epoch = eps_idx

            # beginning of an episode
            state = env.reset()
            state = torch.tensor(state, dtype=torch.float32)
            done = False
            score = 0

            while not done:

                action = self.select_action(state)

                # run one step
                next_state, reward, done, _ = env.step(action)
                next_state = torch.tensor(next_state, dtype=torch.float32)

                self.remember(state, reward)
                state = next_state

                score += reward

            # bookkeeping of stats
            all_scores[eps_idx] = score
            if score > max_score:
                max_score = score

            sys.stdout.write(f"\r [{eps_idx}]: {score}, Avg: {avg:.2f}, Max: {max_score}, Best_avg: {best_avg:.2f}")
            sys.stdout.flush()
            
            if ((eps_idx + 1) % roll_size) == 0:
                avg = np.mean(all_scores[(eps_idx + 1) - roll_size:eps_idx])
                if avg > best_avg:
                    best_avg = avg
                    self.save(eps_idx, "pong_checkpoint_bestavg")

                # print(f"\n [{eps_idx}]: {score}, Avg: {avg:.2f}, Max: {max_score}, Best_avg: {best_avg:.2f}")
                stat_string = f" [{eps_idx}]: {score}, Avg: {avg:.2f}, Max: {max_score}, Best_avg: {best_avg:.2f}\n"
                log(self.log_filename, stat_string)
                self.save(eps_idx, "pong_checkpoint_latest")

                np.save('checkpoints/all_scores.npy', all_scores, allow_pickle=False)
            
            # train every 10 episodes
            if ((eps_idx + 1) % 10) == 0:
                self.update_network()
            
            # graph the scores every 100 eps
            if ((eps_idx + 1) % 100) == 0:
                pass
        
        avg = np.mean(all_scores)
        max_score = np.max(all_scores)
        print(f"\n [{eps_idx}]: {score}, Avg: {avg:.2f}, Max: {max_score}, Best_avg: {best_avg:.2f}")

    def save(self, epoch, path):
        save_dir = 'checkpoints/'
        path = save_dir + path + ".pt"

        Path(save_dir).mkdir(exist_ok=True)

        try:
            lossA = self.lossA
            lossC = self.lossC
        except AttributeError:
            lossA = None
            lossC = None

        torch.save(
            {
                'epoch': epoch,
                'actor_state_dict': self.actor.state_dict(),
                'critic_state_dict': self.critic.state_dict(),
                'optimizerA_state_dict': self.optimizerA.state_dict(),
                'optimizerC_state_dict': self.optimizerC.state_dict(),
                'lossA': lossA,
                'lossC': lossC,
            },
            path
        )

    def load(self, path):
        save_dir = 'checkpoints/'
        path = save_dir + path + ".pt"

        checkpoint = torch.load(path)

        epoch = checkpoint['epoch']
        self.actor.load_state_dict(checkpoint['actor_state_dict'])
        self.critic.load_state_dict(checkpoint['critic_state_dict'])
        self.optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
        self.optimizerC.load_state_dict(checkpoint['optimizerC_state_dict'])

        return epoch

#### Training the agent

In [ ]:
env = gym.make("PongDeterministic-v0")
env = Pong(env)

if Path('logs.txt').exists():
    print("Logs already exist, appending to them.")
agent = A2CAgent(6000, 'logs.txt')
epoch_resume = -1
# epoch_resume = agent.load('pong_checkpoint_bestavg')
agent.learn(env, 1000, 10, epoch_resume)

Resuming from 0, Writing to logs.txt

 [101]: -21.0, Avg: -20.33, Max: -16.0, Best_avg: -20.00

### Part II
### Deep Deterministic Policy Gradients

#### Required Imports

In [ ]:
# derived from https://github.com/wpiszlogin/driver_critic/
import numpy as np
import tensorflow as tf
from tools import *
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

import gymnasium as gym

#### Base Class for the agent

Unlike DQN where we set target network weights to the trainable network weights in defined intervals, we use a smoother update for the target networks' weights. 
![tau_update](https://spinningup.openai.com/en/latest/_images/math/d417987803ca9f61ac60741880a748129bd66dde.svg)

In [2]:
class BaseSolution:
    def __init__(self, action_space, model_outputs=None, noise_mean=None, noise_std=None):

        # Hyperparameters
        self.gamma = 0.99
        self.actor_lr = 0.00001
        self.critic_lr = 0.002
        self.tau = 0.005
        self.memory_capacity = 60000

        # For problems that have specific outputs of an actor model
        self.need_decode_out = model_outputs is not None
        self.model_action_out = model_outputs if model_outputs else action_space.shape[0]
        self.action_space = action_space

        # Init noise generator
        if noise_mean is None:
            noise_mean = np.full(self.model_action_out, 0.0, np.float32)
        if noise_std is None:
            noise_std  = np.full(self.model_action_out, 0.2, np.float32)
        std = self.noise = NoiseGenerator(noise_mean, noise_std)

        # Initialize buffer R
        self.r_buffer = MemoriesRecorder(memory_capacity=self.memory_capacity)

        self.actor_opt      = Adam(self.actor_lr)
        self.critic_opt     = Adam(self.critic_lr)
        self.actor          = None
        self.critic         = None
        self.target_actor   = None
        self.target_critic  = None

    def reset(self):
        self.noise.reset()

    def build_actor(self, state_shape, name="Actor"):
        inputs = layers.Input(shape=state_shape)
        x = inputs
        x = layers.Conv2D(16, kernel_size=(5, 5), strides=(4, 4), padding='valid', use_bias=False, activation="relu")(x)
        x = layers.Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='valid', use_bias=False, activation="relu")(x)
        x = layers.Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='valid', use_bias=False, activation="relu")(x)

        x = layers.Flatten()(x)
        x = layers.Dense(64, activation='relu')(x)
        y = layers.Dense(self.model_action_out, activation='tanh')(x)

        model = Model(inputs=inputs, outputs=y, name=name)
        model.summary()
        return model

    def build_critic(self, state_shape, name="Critic"):
        state_inputs = layers.Input(shape=state_shape)
        x = state_inputs
        x = layers.Conv2D(16, kernel_size=(5, 5), strides=(4, 4), padding='valid', use_bias=False, activation="relu")(x)
        x = layers.Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='valid', use_bias=False, activation="relu")(x)
        x = layers.Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='valid', use_bias=False, activation="relu")(x)

        x = layers.Flatten()(x)
        action_inputs = layers.Input(shape=(self.model_action_out,))
        x = layers.concatenate([x, action_inputs])

        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dense(32, activation='relu')(x)
        y = layers.Dense(1)(x)

        model = Model(inputs=[state_inputs, action_inputs], outputs=y, name=name)
        model.summary()
        return model

    def init_networks(self, state_shape):
        self.actor  = self.build_actor(state_shape)
        self.critic = self.build_critic(state_shape)

        # Build target networks in the same way
        self.target_actor  = self.build_actor(state_shape, name='TargetActor')
        self.target_critic = self.build_critic(state_shape, name='TargetCritic')

        # Copy parameters from action and critic
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())

    def get_action(self, state, add_noise=True):
        prep_state = self.preprocess(state)
        if self.actor is None:
            self.init_networks(prep_state.shape)

        # Get result from a network
        tensor_state = tf.expand_dims(tf.convert_to_tensor(prep_state), 0)
        actor_output = self.actor(tensor_state).numpy()

        # Add noise
        if add_noise:
            actor_output = actor_output[0] + self.noise.generate()
        else:
            actor_output = actor_output[0]

        if self.need_decode_out:
            env_action = self.decode_model_output(actor_output)
        else:
            env_action = actor_output

        # Clip min-max
        env_action = np.clip(np.array(env_action), a_min=self.action_space.low, a_max=self.action_space.high)
        return env_action, actor_output

    def decode_model_output(self, model_out):
        return np.array([model_out[0], model_out[1].clip(0, 1), -model_out[1].clip(-1, 0)])

    def preprocess(self, img, greyscale=False):
        img = img.copy()
        # Remove numbers and enlarge speed bar
        for i in range(88, 93+1):
            img[i, 0:12, :] = img[i, 12, :]

        # Unify grass color
        replace_color(img, original=(102, 229, 102), new_value=(102, 204, 102))

        if greyscale:
            img = img.mean(axis=2)
            img = np.expand_dims(img, 2)

        # Make car black
        car_color = 68.0
        car_area = img[67:77, 42:53]
        car_area[car_area == car_color] = 0

        # Scale from 0 to 1
        img = img / img.max()

        # Unify track color
        img[(img > 0.411) & (img < 0.412)] = 0.4
        img[(img > 0.419) & (img < 0.420)] = 0.4

        # Change color of kerbs
        game_screen = img[0:83, :]
        game_screen[game_screen == 1] = 0.80
        return img

    def learn(self, state, train_action, reward, new_state):
        # Store transition in R
        prep_state     = self.preprocess(state)
        prep_new_state = self.preprocess(new_state)
        self.r_buffer.write(prep_state, train_action, reward, prep_new_state)

        # Sample mini-batch from R
        state_batch, action_batch, reward_batch, new_state_batch  = self.r_buffer.sample()

        state_batch     = tf.convert_to_tensor(state_batch)
        action_batch    = tf.convert_to_tensor(action_batch)
        reward_batch    = tf.convert_to_tensor(reward_batch)
        reward_batch    = tf.cast(reward_batch, dtype=tf.float32)
        new_state_batch = tf.convert_to_tensor(new_state_batch)

        self.update_actor_critic(state_batch, action_batch, reward_batch, new_state_batch)

        # Update target networks
        self.update_target_network(self.target_actor.variables, self.actor.variables)
        self.update_target_network(self.target_critic.variables, self.critic.variables)

    @tf.function
    def update_actor_critic(self, state, action, reward, new_state):
        # Update critic
        with tf.GradientTape() as tape:
            # Calc y
            new_action = self.target_actor(new_state, training=True)
            y = reward + self.gamma * self.target_critic([new_state, new_action], training=True)

            critic_loss = tf.math.reduce_mean(tf.square(y - self.critic([state, action], training=True)))

        critic_gradients = tape.gradient(critic_loss, self.critic.trainable_variables)
        self.critic_opt.apply_gradients(zip(critic_gradients, self.critic.trainable_variables))

        # Update actor
        with tf.GradientTape() as tape:
            critic_out = self.critic([state, self.actor(state, training=True)], training=True)
            actor_loss = -tf.math.reduce_mean(critic_out)  # Need to maximize

        actor_gradients = tape.gradient(actor_loss, self.actor.trainable_variables)
        self.actor_opt.apply_gradients(zip(actor_gradients, self.actor.trainable_variables))

    @tf.function
    def update_target_network(self, target_weights, new_weights):
        for t, n in zip(target_weights, new_weights):
            t.assign((1 - self.tau) * t + self.tau * n)

    def save_solution(self, path='models/'):
        self.actor.save(path + 'actor.h5')
        self.critic.save(path + 'critic.h5')
        self.target_actor.save(path + 'target_actor.h5')
        self.target_critic.save(path + 'target_critic.h5')

    def load_solution(self, path='models/'):
        self.actor = tf.keras.models.load_model(path + 'actor.h5')
        self.critic = tf.keras.models.load_model(path + 'critic.h5')
        self.target_actor = tf.keras.models.load_model(path + 'target_actor.h5')
        self.target_critic = tf.keras.models.load_model(path + 'target_critic.h5')

#### Train the model
Gets an average reward of 400 with 30 mins of training on an Intel i5 7th Gen

In [4]:
# Parameters
n_episodes = 50000
problem = 'CarRacing-v2'

gym.logger.set_level(40)
preview = False
best_result = 0
all_episode_reward = []

In [5]:
# Initialize simulation
env = gym.make(problem)
state, info = env.reset()

In [10]:
# Define custom standard deviation for noise
# We need lesser noise for steering
noise_std = np.array([0.1, 4 * 0.2], dtype=np.float32)
solution = BaseSolution(env.action_space, model_outputs=2, noise_std=noise_std)

2023-03-24 09:44:34.602618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/anudeep/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-24 09:44:34.603680: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-24 09:44:34.603720: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anudeep-ubuntu): /proc/driver/nvidia/version does not exist
2023-03-24 09:44:34.810075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the 

In [13]:
# Loop of episodes
for ie in range(n_episodes):
    state, info = env.reset()
    solution.reset()
    done = False
    episode_reward = 0
    no_reward_counter = 0

    # One-step-loop
    while not done:
        if preview:
            env.render()

        action, train_action = solution.get_action(state)

        # This will make steering much easier
        action /= 4
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Models action output has a different shape for this problem
        solution.learn(state, train_action, reward, new_state)
        state = new_state
        episode_reward += reward

        if reward < 0:
            no_reward_counter += 1
            if no_reward_counter > 200:
                break
        else:
            no_reward_counter = 0

    all_episode_reward.append(episode_reward)
    average_result = np.array(all_episode_reward[-10:]).mean()
    print('Last result:', episode_reward, 'Average results:', average_result)

    if episode_reward > best_result:
        print('Saving best solution')
        solution.save_solution()
        best_result = episode_reward

Model: "Actor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 16)        1200      
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 32)          4608      
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 32)          9216      
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 2)                 130   

Last result: -10.024183006535866 Average results: 44.93864650411285
Last result: -10.224475524475443 Average results: 26.55093916125009
Last result: 19.695857988165315 Average results: 24.837168867978896
Last result: -18.044927536231803 Average results: 16.260749587136758
Last result: -9.896699669966893 Average results: 11.901174710952816
Last result: -7.322292993630489 Average results: 9.154965038869488
Last result: -10.639335180055317 Average results: 6.680677511503886
Last result: -18.255279503105534 Average results: 3.910015620991728
Last result: -3.0946236559139377 Average results: 3.2095516933011616
Last result: -1.4118644067796382 Average results: -6.921782348852962
Last result: 1.7827160493826804 Average results: -5.741092443261106
Last result: -11.76666666666674 Average results: -5.895311557480237
Last result: 52.60565371024762 Average results: -2.6043319852720046
Last result: 19.759414225942024 Average results: 1.1761021909453775
Last result: 58.730894308943796 Average result

Last result: 397.2449275362261 Average results: 80.36334707969546
Saving best solution


Last result: 666.2965986394485 Average results: 147.30246930923172
Saving best solution


Last result: 849.9999999999816 Average results: 232.4436557499078
Saving best solution


Last result: 862.4573378839391 Average results: 318.51111793336344
Saving best solution


Last result: 351.67416974169157 Average results: 354.8552015741993
Last result: 536.3636363636225 Average results: 403.23099983953676
Last result: 421.8978102189742 Average results: 443.44483943883995
Last result: 634.7846416382132 Average results: 501.0502141717669
Last result: 152.61634980988768 Average results: 512.0092988171853
Last result: 81.5971830985928 Average results: 495.4932654930577
Last result: 415.92811501596856 Average results: 497.36158424103195
Last result: 475.4413249211261 Average results: 478.2760568691998
Last result: 616.6666666666601 Average results: 454.9427235358676
Last result: 252.98436578170808 Average results: 393.9954263256445
Last result: 405.2586206896476 Average results: 399.3538714204401
Last result: 446.5945054944993 Average results: 390.3769583335278


KeyboardInterrupt: 

In [6]:
# We can improve stability of solution with noise
noise_mean = np.array([0.0, -0.83], dtype=np.float32)
noise_std = np.array([0.0, 4 * 0.02], dtype=np.float32)
solution = BaseSolution(env.action_space, model_outputs=2, noise_mean=noise_mean, noise_std=noise_std)
solution.load_solution('models/')

2023-03-24 17:00:46.037058: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/anudeep/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-24 17:00:46.042958: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-24 17:00:46.043162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anudeep-ubuntu): /proc/driver/nvidia/version does not exist
2023-03-24 17:00:46.276822: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the 

#### Evaluate the model

In [7]:
n_episodes = 10
env = gym.make(problem, render_mode="human")
state, info = env.reset()
all_episode_reward = []

In [8]:
# Loop of episodes
for ie in range(n_episodes):
    state, info = env.reset()
    solution.reset()
    done = False
    episode_reward = 0
    no_reward_counter = 0

    # One-step-loop
    while not done:

        action, train_action = solution.get_action(state)

        # This will make steering much easier
        action /= 4
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        state = new_state
        episode_reward += reward

        if reward < 0:
            no_reward_counter += 1
            if no_reward_counter > 200:
                break
        else:
            no_reward_counter = 0

    all_episode_reward.append(episode_reward)
    average_result = np.array(all_episode_reward).mean()
    print('Last result:', episode_reward, 'Average results:', average_result)


/home/anudeep/anaconda3/envs/py39/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Last result: 874.3589743589607 Average results: 874.3589743589607


KeyboardInterrupt: 

#### Exercise: Improve the DDPG Agent
1) This code does not use braking efficiently. Find out how and fix it to get a better agent.

2) This code does not make use of the `terminated` flag. Add it to the replay buffer and updates. Check if the model converges faster after doing so.

3) Evaluate the agent without noise.